In [ ]:
# =============================================
# [9/9] 종합 실습 & 배포
# =============================================
# 목표: 학습된 모델을 ONNX로 변환해 보고, FastAPI를 이용한 API 서버 코드를 작성합니다.
# (이 노트북은 Colab에서 직접 실행하는 부분과, 로컬에서 실행할 코드 예시로 구성됩니다)

# --- 1. ONNX 변환 및 추론 (Colab에서 실행 가능) ---
!pip install transformers torch onnx onnxruntime

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Step 7에서 파인튜닝한 모델이 있다고 가정. 여기서는 원본 모델로 대체.
model_name = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()

# 더미 입력 생성
dummy_text = "이것은 ONNX 변환을 위한 샘플 문장입니다."
dummy_input = tokenizer(dummy_text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
onnx_model_path = "bert_classifier.onnx"

# torch.onnx.export로 모델 변환
torch.onnx.export(
    model,
    (dummy_input['input_ids'], dummy_input['attention_mask']),
    onnx_model_path,
    input_names=['input_ids', 'attention_mask'],
    output_names=['logits'],
    dynamic_axes={'input_ids': {0: 'batch_size'}, 'attention_mask': {0: 'batch_size'}, 'logits': {0: 'batch_size'}},
    opset_version=11
)

print(f"모델이 {onnx_model_path}에 저장되었습니다.")

# ONNX Runtime으로 추론
import onnxruntime as ort
import numpy as np

ort_session = ort.InferenceSession(onnx_model_path)

# PyTorch 추론 결과와 비교
with torch.no_grad():
    torch_logits = model(**dummy_input).logits.numpy()

# ONNX 추론
ort_inputs = {ort_session.get_inputs()[0].name: dummy_input['input_ids'].numpy(),
              ort_session.get_inputs()[1].name: dummy_input['attention_mask'].numpy()}
ort_logits = ort_session.run(None, ort_inputs)[0]

# 결과 비교
np.testing.assert_allclose(torch_logits, ort_logits, rtol=1e-03, atol=1e-05)
print("\nONNX 추론 결과가 PyTorch 결과와 일치합니다!")

# --- 2. FastAPI 서버 구현 (로컬 실행용 코드) ---
print("\n--- 아래는 로컬 환경에서 실행할 FastAPI 서버 코드 예시입니다. ---")

fastapi_code = """
from fastapi import FastAPI
from pydantic import BaseModel
import onnxruntime as ort
from transformers import AutoTokenizer
import numpy as np

# FastAPI 앱 초기화
app = FastAPI()

# 모델 및 토크나이저 로드 (앱 시작 시 한 번만)
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
ort_session = ort.InferenceSession("bert_classifier.onnx")

# 요청 Body 모델 정의
class Item(BaseModel):
    text: str

@app.post("/predict/")
async def predict(item: Item):
    # 1. 입력 텍스트 토크나이징
    inputs = tokenizer(
        item.text, return_tensors="np", 
        padding="max_length", truncation=True, max_length=128
    )
    ort_inputs = {
        'input_ids': inputs['input_ids'].astype(np.int64),
        'attention_mask': inputs['attention_mask'].astype(np.int64)
    }

    # 2. ONNX 런타임으로 추론
    ort_logits = ort_session.run(None, ort_inputs)[0]
    
    # 3. 결과 처리
    prediction = int(np.argmax(ort_logits, axis=1)[0])
    probability = float(np.exp(ort_logits).sum(axis=1)[0])
    
    return {"prediction": prediction, "text": item.text}

# 실행 방법 (터미널):
# 1. pip install fastapi uvicorn python-multipart
# 2. 위 코드를 main.py로 저장
# 3. uvicorn main:app --reload
"""
print(fastapi_code)


# --- 3. Docker 이미지화 (로컬 실행용 파일 예시) ---
print("\n--- 아래는 FastAPI 앱을 Docker로 패키징하기 위한 파일 예시입니다. ---")

dockerfile_content = """
# 1. 베이스 이미지 선택
FROM python:3.9-slim

# 2. 작업 디렉토리 설정
WORKDIR /app

# 3. 의존성 파일 복사 및 설치
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# 4. 앱 소스코드와 모델 파일 복사
COPY ./main.py .
COPY ./bert_classifier.onnx .

# 5. 포트 노출
EXPOSE 8000

# 6. 앱 실행 명령어
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]
"""

requirements_content = """
fastapi
uvicorn
onnxruntime
transformers
torch
numpy
"""

print("\n[Dockerfile 내용]")
print(dockerfile_content)
print("\n[requirements.txt 내용]")
print(requirements_content)
print("\n실행 명령어 (터미널):")
print("docker build -t nlp-api .")
print("docker run -d -p 8000:8000 nlp-api")